In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

c:\Users\ericy\anaconda3\envs\machine_learning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

LLM is explicitly disabled. Using MockLLM.


In [6]:
documents = SimpleDirectoryReader("Archive").load_data()

In [7]:
index = VectorStoreIndex.from_documents(documents)

c:\Users\ericy\anaconda3\envs\machine_learning\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [8]:
top_k = 5

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [9]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [12]:
query = "How is attention retained?"
response = query_engine.query(query)

In [13]:
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context:
Th  is chapter focuses on the mechanisms of selective att  ention and its role in perception and awareness.
TAKE-HOME MESSAGES
 ■Selective attention is the ability to focus awareness on 
one stimulus, thought, or action while ignoring other, ir-relevant stimuli, thoughts, and actions.
 ■Arousal is a global physiological and psychological brain state, whereas selective attention describes what we attend and ignore within any specifi  c level (high versus low) of arousal.
 ■Specifi  c networks for the control of attention include cortical and subcortical structures.
 ■Attention infl  uences how we process sensory inputs, store that information in memory, process it semanti-cally, and act on it.

1 Selective Attention and the Anatomy of Attention 276
7 .2 The Neuropsychology of Attention 
278
Neglect 278
Comparing Neglect and Bálint’s 
Syndrome 281
7 .3 Models of Attention 282
Hermann von Helmholtz and Covert Attention 
282
The Cocktail Party Effect 283
Early-Selection Models Ver

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("./Wizard-Vicuna-30B-Uncensored-GPTQ", device_map='cuda')
model = AutoModelForCausalLM.from_pretrained("./Wizard-Vicuna-30B-Uncensored-GPTQ", device_map='cuda')
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

system = "You are a professor of psychology who eagerly answers the user's questions and backs up claims using citations and context given. You provide long, detailed and eloquent explanations that are easy to understand"
input_text = "SYSTEM: " + system
in_length = 0

def inference(input_text):
    global in_length
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    in_length = len(input_text)
    outputs = model.generate(**inputs, max_length = 4096)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def output(generated_text):
    global in_length
    print("ASSISTANT: " + generated_text[in_length+1:])
    print("\n")

def get_query(query):
    response = query_engine.query(query)
    context = "Context:\n"
    for i in range(top_k):
        context = context + response.source_nodes[i].text + "\n\n"
    return(context)


print("\n" * 50)    

while True:
    user = input("USER: ")
    context = get_query(user)
    print("\n")
    input_text += " USER:" + context + "\n" + user + " ASSISTANT: "
    generated_text = inference(input_text)
    output(generated_text)
    input_text = generated_text

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
c:\Users\ericy\anaconda3\envs\machine_learning\Lib\site-packages\transformers\modeling_utils.py:4713: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at ./Wizard-Vicuna-30B-Uncensored-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'